<a href="https://colab.research.google.com/github/Ananya-AJ/SentimentAnalysis/blob/main/sentiments_amazonreviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import pandas as pd
import numpy as np

In [ ]:
unpickled_df = pd.read_pickle("/content/Copy of cell_phone_reviews_df.pkl") 

In [ ]:
unpickled_df.head()

In [ ]:
unpickled_df.isna().sum()

overall                 0
verified                0
reviewTime              0
reviewerID              0
asin                    0
style              523196
reviewerName          135
reviewText            765
summary               517
unixReviewTime          0
vote              1036403
image             1101330
dtype: int64

In [ ]:
review_df = unpickled_df.dropna(subset=['reviewText'])

In [ ]:
review_df.isna().sum()

overall                 0
verified                0
reviewTime              0
reviewerID              0
asin                    0
style              522865
reviewerName          134
reviewText              0
summary               466
unixReviewTime          0
vote              1035671
image             1100659
dtype: int64

In [ ]:
import re
import string

In [ ]:
review_df["reviewText"] = review_df['reviewText'].astype(str).str.lower()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
review_df.head()

In [ ]:
review_df['reviewText'] = review_df['reviewText'].astype(str).replace('^a-zA-Z','')

In [ ]:
review_df.head()

In [ ]:
!pip install nltk

In [ ]:
from nltk.tokenize import RegexpTokenizer

regexp = RegexpTokenizer('\w+')

review_df['review_token']=review_df['reviewText'].apply(regexp.tokenize)
review_df.head(3)

In [ ]:
import nltk

nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

stopwords = nltk.corpus.stopwords.words("english")
my_stopwords = ['https']
stopwords.extend(my_stopwords)

In [ ]:
review_df['review_token'] = review_df['review_token'].apply(lambda x: [item for item in x if item not in stopwords])
review_df.head(3)

In [ ]:
review_df['review_string'] = review_df['review_token'].apply(lambda x: ' '.join([item for item in x if len(item)>2]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
review_df[['reviewText', 'review_token', 'review_string']].head()

,reviewText,review_token,review_string
0,looks even better in person. be careful to not...,"[looks, even, better, person, careful, drop, p...",looks even better person careful drop phone of...
1,when you don't want to spend a whole lot of ca...,"[want, spend, whole, lot, cash, want, great, d...",want spend whole lot cash want great deal shop...
2,"so the case came on time, i love the design. i...","[case, came, time, love, design, actually, mis...",case came time love design actually missing st...
3,don't care for it. gave it as a gift and they...,"[care, gave, gift, okay, expected]",care gave gift okay expected
4,"i liked it because it was cute, but the studs ...","[liked, cute, studs, fall, easily, protect, ph...",liked cute studs fall easily protect phone wou...


In [ ]:
all_words = ' '.join([word for word in review_df['review_string']])

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
tokenized_words = nltk.tokenize.word_tokenize(all_words)

frequency dictionary for each word

In [ ]:
from nltk.probability import FreqDist

fdist = FreqDist(tokenized_words)
fdist

FreqDist({'phone': 707591, 'case': 641982, 'great': 329624, 'one': 280474, 'screen': 271849, 'good': 257063, 'like': 243741, 'product': 196465, 'well': 194075, 'use': 179899, ...})

In [ ]:
review_df['review_fdist'] = review_df['review_token'].apply(lambda x: ' '.join([item for item in x if fdist[item] >= 1 ]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
review_analysis = review_df[['reviewText','review_token','review_string','review_fdist']]

In [ ]:
review_analysis.head()

#lemmitization

In [2]:
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [3]:
# from nltk.stem import WordNetLemmatizer

# wordnet_lem = WordNetLemmatizer()

# review_analysis['review_lemma'] = review_analysis['review_fdist'].apply(wordnet_lem.lemmatize)

In [ ]:
review_analysis.head(3)

In [ ]:
review_analysis['is_equal']= (review_analysis['review_fdist']==review_analysis['review_lemma'])

In [ ]:
review_analysis.is_equal.value_counts()

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

In [ ]:
review_analysis['polarity'] = review_analysis['review_lemma'].apply(lambda x: analyzer.polarity_scores(x))
review_analysis.tail(3)

In [ ]:
# analyzer = SentimentIntensityAnalyzer()
# review_analysis['compound'] = [analyzer.polarity_scores(x)['compound'] for x in review_analysis['review_lemma']]
# review_analysis['neg'] = [analyzer.polarity_scores(x)['neg'] for x in review_analysis['review_lemma']]
# review_analysis['neu'] = [analyzer.polarity_scores(x)['neu'] for x in review_analysis['review_lemma']]
# review_analysis['pos'] = [analyzer.polarity_scores(x)['pos'] for x in review_analysis['review_lemma']]

In [ ]:
review_analysis.head()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
review_df = pd.read_csv('/content/drive/My Drive/256vaderfiles/review_analysis_scores.csv')

In [ ]:
review_df.head()

In [ ]:
review_df['sentiment'] = review_df['compound'].apply(lambda x: 1 if x >=0 else 0)
review_df.head(4)

##Use VADER to create labels

In [ ]:
review_df.shape

In [ ]:
X = review_df.drop(['neg','neu','pos','sentiment','compound'],axis=1)

In [ ]:
y = review_df[['sentiment']]

In [ ]:
X.review_lemma = X.review_lemma.astype(str)


In [ ]:
phrase = np.array(X['review_lemma'])
sentiments = np.array(y['sentiment'])

In [ ]:
!pip install sklearn

In [ ]:
from sklearn.model_selection import train_test_split
phrase_train, phrase_test,sentiments_train, sentiments_test = train_test_split(phrase,sentiments ,
                                   random_state=104, 
                                   test_size=0.33)

In [ ]:
# for i in range(0,len(X_train)-1):

#     if type(X_train.iloc[i]['review_lemma']) != str:
#         print(i)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = CountVectorizer(stop_words='english',max_features=10000)
cv_train_features = cv.fit_transform(phrase_train)

In [ ]:
cv_train_features.shape

In [ ]:
cv_test_features =  cv.transform(phrase_test)

In [ ]:
cv_test_features.shape

Using tf-idf to build features

In [ ]:
tv = TfidfVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1,2),
                     sublinear_tf=True)
tv_train_features = tv.fit_transform(phrase_train)

In [ ]:
tv_test_features = tv.transform(phrase_test)

In [ ]:
tv_train_features.shape

In [ ]:
from sklearn.linear_model import SGDClassifier, LogisticRegression

lr = LogisticRegression(penalty='l2', max_iter=100, C=1)

In [ ]:
def train_predict_model(classifier, 
                        train_features, train_labels, 
                        test_features, test_labels):
    # build model    
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features) 
    return predictions 

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc 
def get_metrics(true_labels, predicted_labels):
    
    print('Accuracy:', np.round(
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        4))
    print('Precision:', np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('Recall:', np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('F1 Score:', np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
                        

In [ ]:
lr_tfidf_predictions = train_predict_model(classifier=lr, 
                                               train_features=tv_train_features, train_labels=sentiments_train,
                                               test_features=tv_test_features, test_labels=sentiments_test)


In [ ]:
get_metrics(true_labels=sentiments_test, predicted_labels=lr_tfidf_predictions)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 50, random_state = 42)
rf.fit(tv_train_features, sentiments_train)
predictions = rf.predict(tv_test_features)


In [ ]:
import pickle
pickle.dump(rf, open('rfmodel.pkl', 'wb'))

In [ ]:
get_metrics(true_labels=sentiments_test, predicted_labels=lr_tfidf_predictions)